The aim of this experiment is to see how Markov-Snippets THUG behaves compared to SMC-THUG on the G and K problem.

In [47]:
import numpy as np
from numpy.random import rand, randn
from numpy import ones, exp, log, diag, vstack, pi, array, r_, isfinite, logspace, zeros, eye
from numpy.linalg import norm, solve
from numpy.random import default_rng, choice
from scipy.optimize import fsolve
from scipy.stats import multivariate_normal as MVN
from scipy.special import ndtri, ndtr
from scipy.stats import uniform as udist
from scipy.stats import norm as ndist
import scipy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from ipywidgets.widgets import interact
import math
from warnings import catch_warnings, filterwarnings
import time

from RWM import RWM
from Manifolds.Manifold import Manifold
from tangential_hug_functions import HugTangentialMultivariate
import arviz
from utils import ESS_univariate
from seaborn import kdeplot

from copy import deepcopy
from matplotlib import rc
import seaborn as sns

# G and K functions and settings

In [27]:
class GKManifold(Manifold):
    def __init__(self, ystar):
        self.m = len(ystar)            # Number constraints = dimensionality of the data
        self.d = 4                     # Manifold has dimension 4 (like the parameter θ)
        self.n = self.d + self.m       # Dimension of ambient space is m + 4
        self.ystar = ystar
        # N(0, 1) ---> U(0, 10).
        self.G    = lambda θ: 10*ndtr(θ)
        # U(0, 10) ---> N(0, 1)
        self.Ginv = lambda θ: ndtri(θ/10)

    def q(self, ξ):
        """Constraint for G and K."""
        ξ = r_[self.G(ξ[:4]), ξ[4:]]   # expecting theta part to be N(0, 1)
        with catch_warnings():
            filterwarnings('error')
            try:
                return (ξ[0] + ξ[1]*(1 + 0.8*(1 - exp(-ξ[2]*ξ[4:]))/(1 + exp(-ξ[2]*ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3])*ξ[4:]) - self.ystar
            except RuntimeWarning:
                raise ValueError("Constraint found Overflow warning.")
                
    def _q_raw_uniform(self, ξ):
        """Constraint function expecting ξ[:4] ~ U(0, 10). It doesn't do any warning check."""
        return (ξ[0] + ξ[1]*(1 + 0.8*(1 - exp(-ξ[2]*ξ[4:]))/(1 + exp(-ξ[2]*ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3])*ξ[4:]) - self.ystar
    def _q_raw_normal(self, ξ):
        """Same as `_q_raw_uniform` except expects ξ[:4]~N(0,1)."""
        ξ = r_[self.G(ξ[:4]), ξ[4:]] 
        return self._q_raw_uniform(ξ)

    def Q(self, ξ):
        """Transpose of Jacobian for G and K. """
        ξ = r_[self.G(ξ[:4]), ξ[4:]]
        return vstack((
        ones(len(ξ[4:])),
        (1 + 0.8 * (1 - exp(-ξ[2] * ξ[4:])) / (1 + exp(-ξ[2] * ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3]) * ξ[4:],
        8 * ξ[1] * (ξ[4:]**2) * ((1 + ξ[4:]**2)**ξ[3]) * exp(ξ[2]*ξ[4:]) / (5 * (1 + exp(ξ[2]*ξ[4:]))**2),
        ξ[1]*ξ[4:]*((1+ξ[4:]**2)**ξ[3])*(1 + 9*exp(ξ[2]*ξ[4:]))*log(1 + ξ[4:]**2) / (5*(1 + exp(ξ[2]*ξ[4:]))),
        diag(ξ[1]*((1+ξ[4:]**2)**(ξ[3]-1))*(((18*ξ[3] + 9)*(ξ[4:]**2) + 9)*exp(2*ξ[2]*ξ[4:]) + (8*ξ[2]*ξ[4:]**3 + (20*ξ[3] + 10)*ξ[4:]**2 + 8*ξ[2]*ξ[4:] + 10)*exp(ξ[2]*ξ[4:]) + (2*ξ[3] + 1)*ξ[4:]**2 + 1) / (5*(1 + exp(ξ[2]*ξ[4:]))**2))
    ))
    
    def J(self, ξ):
        """Safely computes Jacobian."""
        with catch_warnings():
            filterwarnings('error')
            try:
                return self.Q(ξ).T
            except RuntimeWarning:
                raise ValueError("J computation found Runtime warning.")
                
    def fullJacobian(self, ξ):
        """J_f(G(ξ)) * J_G(ξ)."""
        JGbar = la.block_diag(10*np.diag(ndist.pdf(ξ[:4])), eye(len(ξ[4:])))
        return self.J(ξ) @ JGbar
                
    def log_parameter_prior(self, θ):
        """IMPORTANT: Typically the prior distribution is a U(0, 10) for all four parameters.
        We keep the same prior but since we don't want to work on a constrained space, we 
        reparametrize the problem to an unconstrained space N(0, 1)."""
        with catch_warnings():
            filterwarnings('error')
            try:
                return udist.logpdf(self.G(θ), loc=0.0, scale=10.0).sum() + ndist.logpdf(θ).sum()
            except RuntimeWarning:
                return -np.inf
            
    def logprior(self, ξ):
        """Computes the prior distribution for G and K problem. Notice this is already reparametrized."""
        return self.log_parameter_prior(ξ[:4]) - ξ[4:]@ξ[4:]/2

    def logη(self, ξ):
        """log posterior for c-rwm. This is on the manifold."""
        try:
            J = self.J(ξ)
            logprior = self.logprior(ξ)
            correction_term  = - math.prod(np.linalg.slogdet(J@J.T))/2 
            return  logprior + correction_term
        except ValueError as e:
            return -np.inf
        
    def generate_logηϵ(self, ϵ, kernel='normal'):
        """Returns the log abc posterior for THUG."""
        if kernel not in ['normal']:
            raise NotImplementedError
        else:
            def log_abc_posterior(ξ):
                """Log-ABC-posterior."""
                u = self.q(ξ)
                m = len(u)
                return self.logprior(ξ) - u@u/(2*ϵ**2) - m*log(ϵ) - m*log(2*pi)/2
            return log_abc_posterior
            
    def logp(self, v):
        """Log density for normal on the tangent space."""
        return MVN(mean=zeros(self.d), cov=eye(self.d)).logpdf(v)
    
    def is_on_manifold(self, ξ, tol=1e-8):
        """Checks if ξ is on the ystar manifold."""
        return np.max(abs(self.q(ξ))) < tol
    
    
"""
OTHER FUNCTIONS
"""    
def generate_powers_of_ten(max_exponent, min_exponent):
    """E.g. generate_powers_of_ten(2, -1) will return 100, 10, 0, 0.1."""
    number_of_powers = max_exponent + abs(min_exponent) + 1
    return logspace(start=max_exponent, stop=min_exponent, num=number_of_powers, endpoint=True)


def data_generator(θ0, m, seed):
    """Stochastic Simulator. Generates y given θ."""
    rng = default_rng(seed)
    z = rng.normal(size=m)
    ξ = r_[θ0, z]
    return ξ[0] + ξ[1]*(1 + 0.8*(1 - exp(-ξ[2]*ξ[4:]))/(1 + exp(-ξ[2]*ξ[4:]))) * ((1 + ξ[4:]**2)**ξ[3])*ξ[4:]

def find_point_on_manifold(ystar, ϵ, max_iter=1000, tol=1.49012e-08):
    """Find a point on the data manifold."""
    i = 0
    manifold = GKManifold(ystar=ystar)
    log_abc_posterior = manifold.generate_logηϵ(ϵ)
    with catch_warnings():
        filterwarnings('error')
        while i <= max_iter:
            i += 1
            try: 
                # Sample θ from U(0, 10)
                θfixed = randn(4)
                function = lambda z: manifold._q_raw_normal(r_[θfixed, z])
                z_guess  = randn(manifold.m)
                z_found  = fsolve(function, z_guess, xtol=tol)
                ξ_found  = r_[θfixed, z_found]
                if not isfinite([log_abc_posterior(ξ_found)]):
                    pass
                else:
                    return ξ_found

            except RuntimeWarning:
                continue
        raise ValueError("Couldn't find a point, try again.") 
        
        
def find_point_on_manifold_from_θ(ystar, θfixed, ϵ, maxiter=2000, tol=1.49012e-08):
    """Same as the above but we provide the θfixed. Can be used to find a point where
    the theta is already θ0."""
    i = 0
    manifold = GKManifold(ystar=ystar)
    log_abc_posterior = manifold.generate_logηϵ(ϵ)
    function = lambda z: manifold._q_raw_normal(r_[θfixed, z])
    with catch_warnings():
        filterwarnings('error')
        while i <= maxiter:
            i += 1
            try:
                z_guess  = randn(manifold.m)
                z_found  = fsolve(function, z_guess, xtol=tol)
                ξ_found  = r_[θfixed, z_found]
                if not isfinite([log_abc_posterior(ξ_found)]):
                    raise ValueError("Couldn't find a point.")
                else:
                    return ξ_found
            except RuntimeWarning:
                continue
        raise ValueError("Couldn't find a point, try again.")



In [68]:
def generate_setting(m, ϵs, B, δ, N, thinning=10):
    """Generates an object from which one can grab the settings. This allows one to run multiple scenarios."""
    θ0        = array([3.0, 1.0, 2.0, 0.5])      # True parameter value on U(0, 10) scale.
    d         = 4 + m                            # Dimensionality of ξ=(θ, z)
    ystar     = data_generator(θ0, m, seed=1234) # Observed data
    q         = MVN(zeros(d), eye(d))            # Proposal distribution for THUG
    ξ0        = find_point_on_manifold_from_θ(ystar=ystar, θfixed=ndtri(θ0/10), ϵ=1e-5, maxiter=5000, tol=1e-15)
    manifold  = GKManifold(ystar)
    return {
        'θ0': θ0,
        'm' : m,
        'd' : d,
        'ystar': ystar,
        'q': q,
        'ξ0': ξ0,
        'ϵs': ϵs,
        'B': B,
        'δ': δ,
        'N': N,
        'manifold': manifold,
        'thinning': thinning
    }

# Multivariate Markov-Snippets

In [69]:
def linear_project(v, J):
    """Projects by solving linear system."""
    return J.T.dot(solve(J.dot(J.T), J.dot(v)))

def THUGIntegratorMultivariate(z0, B, δ):
    """THUG Integrator for the 2D example (ie using gradients, not jacobians)."""
    trajectory = zeros((B + 1, len(z0)))
    x0, v0 = z0[:len(z0)//2], z0[len(z0)//2:]
    x, v = x0, v0
    trajectory[0, :] = z0
    # Integrate
    for b in range(B):
        x = x + δ*v/2
        v = v - 2*linear_project(v, SETTINGS100['manifold'].fullJacobian(x))
        x = x + δ*v/2
        trajectory[b+1, :] = np.hstack((x, v))
    return trajectory

def generate_THUGIntegratorMultivariate(B, δ):
    """Returns a THUG integrator for a given B and δ."""
    integrator = lambda z: THUGIntegratorMultivariate(z, B, δ)
    return integrator


#### Metropolis-Hastings version for SMC version
def THUG_MH(z0, B, δ, logpi):
    """Similar to THUGIntegratoUnivariateOnlyEnd but this uses a MH step."""
    x0, v0 = z0[:len(z0)//2], z0[len(z0)//2:]
    x, v = x0, v0
    logu = np.log(np.random.rand())
    for _ in range(B):
        x = x + δ*v/2
        v = v - 2*linear_project(v, SETTINGS100['manifold'].fullJacobian(x))
        x = x + δ*v/2
    if logu <= logpi(x) - logpi(x0):
        # accept new point
        return np.concatenate((x, v))
    else:
        # accept old point
        return z0

In [89]:
class MultivariateMarkovSnippetsTHUG:
    
    def __init__(self, SETTINGS):
        """Multivariate Markov Snippets SMC samplers corresponding exactly to Algorithm 1 in Christophe's notes.
        It uses the Multivariate THUG kernel as its mutation kernel. The sequence of distributions is fixed here 
        since we provide ϵs, i.e. a list of tolerances which automatically fully specify the posterior 
        distributions used at each round.
        
        Parameters
        ----------
        
        :param N: Number of particles
        :type N:  int
        
        :param B: Number of bounces for the THUG integrator. Equivalent to `L` Leapfrog steps in HMC.
        :type B: int
        
        :param δ: Step-size used at each bounce, for the THUG integrator.
        :type δ: float
        
        :param d: Dimensionality of the `x` component of each particle, and equally dimensionality of 
                  `v` component of each particle. Therefore each particle has dimension `2d`.
        
        :param ϵs: Tolerances that fully specify the sequence of target filamentary distributions.
        :type ϵs: iterable
        """
        # Input variables
        self.N  = SETTINGS['N']       
        self.B  = SETTINGS['B']
        self.δ  = SETTINGS['δ']
        self.d  = SETTINGS['d']
        self.ϵs = SETTINGS['ϵs']    
        self.manifold = SETTINGS['manifold']
        self.SETTINGS = SETTINGS
        
        # Variables derived from the above
        self.P  = len(self.ϵs) - 1                                            # Number of target distributions
        self.log_ηs = [self.manifold.generate_logηϵ(ϵ) for ϵ in self.ϵs] # List of filamentary distributions 
        self.ψ = generate_THUGIntegratorMultivariate(self.B, self.δ)
    
    def initialize_particles(self):
        """Initialize by sampling with RWM on distribution with ϵ0."""
        # Initialize first position on the manifold
        x0 = self.SETTINGS['ξ0']
        # Sample using RWM
        burn_in = 100
        thinning = 10
        TO_BE_THINNED, acceptance = RWM(x0, self.δ, burn_in + thinning*self.N, self.log_ηs[0])
        print("Initializing particles. Acceptance: ", np.mean(acceptance)*100)
        # Thin the samples to obtain the particles
        initialized_particles = TO_BE_THINNED[burn_in:][::thinning]
        # Refresh velocities and form particles
        v0 = np.random.normal(loc=0.0, scale=1.0, size=(self.N, self.d))
        z0 = np.hstack((initialized_particles, v0))
        self.starting_particles = z0
        return z0
        
    def sample(self):
        """Starts the Markov Snippets sampler."""
        starting_time = time.time() 
        N = self.N
        B = self.B
        ## Storage
        #### Store z_n^{(i)}
        self.ZN  = np.zeros((self.P+1, N, 2*self.d))
        #### Store z_{n, k}^{(i)} so basically all the N(T+1) particles
        self.ZNK  = np.zeros((self.P, N*(B+1), 2*self.d))
        self.Wbar = np.zeros((self.P, N*(B+1)))
        self.ESS  = np.zeros((self.P))
        self.K_RESAMPLED = zeros((self.P, self.N))
        # Initialize particles
        z = self.initialize_particles()   # (N, 2d)
        self.ZN[0] = z
        # For each target distribution, run the following loop
        for n in range(1, self.P+1):
            print("### Iteration: ", n, " ϵ = ", self.ϵs[n])
            # Compute trajectories
            Z = np.apply_along_axis(self.ψ, 1, z) # should have shape (N, B+1, 2d)
            self.ZNK[n-1] = Z.reshape(N*(B+1), 2*self.d)
            # Compute weights.
            #### Log-Denominator: shared for each point in the same trajectory
            log_μnm1_z  = np.apply_along_axis(self.log_ηs[n-1], 1, Z[:, 0, :self.d])  # (N, )
            log_μnm1_z  = np.repeat(log_μnm1_z, self.B+1, axis=0).reshape(N, B+1)     # (N, B+1)
            #### Log-Numerator: different for each point on a trajectory.
            log_μn_ψk_z = np.apply_along_axis(self.log_ηs[n], 2, Z[:, :, :self.d])    # (N, B+1)
            #### Put weights together
            W = exp(log_μn_ψk_z - log_μnm1_z) #np.exp(log_μn_ψk_z - log_μnm1_z)
            print("W nan: ", np.isnan(W).sum())
            print("W sum: ", W.sum())
            print("W min: ", W.min())
            print("W med: ", np.median(W))
            print("W max: ", W.max())
            #### Normalize weights
            W = W / W.sum()
            # store weights (remember these are \bar{w})
            self.Wbar[n-1] = W.flatten()
            # compute ESS
            self.ESS[n-1] = 1 / np.sum(W**2)
            print("ESS: ", self.ESS[n-1])
            # Resample down to N particles
            resampling_indeces = choice(a=np.arange(N*(B+1)), size=N, p=W.flatten())
            print("Unique Indeces: ", len(np.unique(resampling_indeces)))
            unravelled_indeces = np.unravel_index(resampling_indeces, (N, B+1))
            self.K_RESAMPLED[n-1] = unravelled_indeces[1]
            indeces = np.dstack(unravelled_indeces).squeeze()
            z = np.vstack([Z[tuple(ix)] for ix in indeces])     # (N, 2d)
            
            # Rejuvenate velocities of N particles
            z[:, self.d:] = np.random.normal(loc=0.0, scale=1.0, size=(N, self.d))
            self.ZN[n] = z
        self.total_time = time.time() - starting_time
        return z

In [100]:
np.apply_along_axis(MSTHUG.ψ, 1, MSTHUG.starting_particles).shape

(1000, 21, 108)

In [90]:
# Settings
ϵs = [10.0, 5.0, 2.0, 1.5, 1.0, 0.75, 0.5, 0.25, 0.1, 0.07] #np.geomspace(start=1.0, stop=1e-6, num=20)
SETTINGS50 = generate_setting(m=50, ϵs=ϵs, B=20, δ=0.01, N=1000, thinning=20)
# Instantitate the algorithm
MSTHUG = MultivariateMarkovSnippetsTHUG(SETTINGS50)
# Sample
zP = MSTHUG.sample()

Initializing particles. Acceptance:  65.78217821782178
### Iteration:  1  ϵ =  5.0
W nan:  0
W sum:  3.45756072943243e+16
W min:  29426.48701357718
W med:  474678010603.9958
W max:  158831845252365.28
ESS:  2470.424331823293
Unique Indeces:  862
### Iteration:  2  ϵ =  2.0
W nan:  0
W sum:  137015689798825.6
W min:  7.426448642393035e-17
W med:  101.21397756528955
W max:  1488040911023.8955
ESS:  255.270732790487
Unique Indeces:  362
### Iteration:  3  ϵ =  1.5
W nan:  0
W sum:  629.5153263192589
W min:  7.176933172726405e-07
W med:  0.02433554191135299
W max:  0.9105854467307908
ESS:  6954.364949233977
Unique Indeces:  929
### Iteration:  4  ϵ =  1.0
W nan:  0
W sum:  7.497453293768899e-10
W min:  6.801253816976773e-18
W med:  2.961926950023106e-14
W max:  2.753043754789736e-12
ESS:  6183.3439564296295
Unique Indeces:  932
### Iteration:  5  ϵ =  0.75
W nan:  0
W sum:  2.7339282412411065e-21
W min:  1.2340548756615851e-28
W med:  1.0324824984483028e-25
W max:  9.517393083508094e-24
ES

/var/folders/31/dthljx8x5mldm2ll5_zz7xrr0000gn/T/ipykernel_63090/3557207091.py:94: RuntimeWarning: invalid value encountered in true_divide
  W = W / W.sum()


ValueError: probabilities contain NaN

In [ ]:
def plot_histogram(n):
    fig, ax = plt.subplots(figsize=(20, 4))
    _, bins, _ = ax.hist(MSTHUG.K_RESAMPLED[n, :], density=True, bins=SETTINGS50['B'], edgecolor='k', color='lightsalmon')
    ax.set_xticks(bins)
    ax.set_xticklabels(bins.astype(int))
    return plt.show()
    
interact(plot_histogram, n=(0, len(ϵs) - 2))

# SMC

In [43]:
class MultivariateMarkovSnippetsTHUGMetropolised:
    
    def __init__(self, SETTINGS):
        """Metropolised version: for each particle compute the endpoint of trajectory and its weight.
        If the weight is positive, we accept the final point, otherwise we accept the initial point. 
        
        Parameters
        ----------
        
        :param N: Number of particles
        :type N:  int
        
        :param B: Number of bounces for the THUG integrator. Equivalent to `L` Leapfrog steps in HMC.
        :type B: int
        
        :param δ: Step-size used at each bounce, for the THUG integrator.
        :type δ: float
        
        :param d: Dimensionality of the `x` component of each particle, and equally dimensionality of 
                  `v` component of each particle. Therefore each particle has dimension `2d`.
        
        :param ϵs: Tolerances that fully specify the sequence of target filamentary distributions.
        :type ϵs: iterable
        """
        # Input variables
        self.N  = SETTINGS['N']       
        self.δ  = SETTINGS['δ']
        self.d  = SETTINGS['d']
        self.ϵs = SETTINGS['ϵs']
        self.manifold = SETTINGS['manifold']
        self.SETTINGS = SETTINGS
        self.B = SETTINGS['B']
        
        # Variables derived from the above
        self.P  = len(self.ϵs) - 1                                       # Number of target distributions
        self.log_ηs = [self.manifold.generate_logηϵ(ϵ) for ϵ in self.ϵs]     # List of filamentary distributions 
        
    
    def initialize_particles(self):
        """To initialize particles, we sample from a uniform on a large rectangle.
        A rectangle of size [-100, 100] should be plenty large."""
        # Initialize particles by sampling from η_ϵ0 for a large ϵ0 which can be given as an argument.
        # Sample a point from the prior
        x0 = self.SETTINGS['ξ0']
        # Use RWM starting from x0
        burn_in = 100
        TO_BE_THINNED, _ = RWM(x0, self.δ, burn_in + 10*self.N, self.log_ηs[0])
        # Thin the samples to obtain the particles
        initialized_particles = TO_BE_THINNED[burn_in:][::10]
        v0 = np.random.normal(loc=0.0, scale=1.0, size=(self.N, self.d))
        z0 = np.hstack((initialized_particles, v0))
        self.starting_particles = z0
        return z0
        
    def sample(self):
        """Starts the Markov Snippets sampler."""
        starting_time = time.time()
        # Initialize particles
        z = self.initialize_particles()   # (N, 2d)
        # Storage
        self.PARTICLES    = zeros((self.P+1, self.N, 2*self.d))
        self.PARTICLES[0] = z
        self.WEIGHTS      = zeros((self.P+1, self.N))
        self.WEIGHTS[0]   = 1 / self.N
        self.ESS          = zeros(self.P+1)
        self.ESS[0]       = 1 / np.sum(self.WEIGHTS[0]**2)
        # For each target distribution, run the following loop
        for n in range(1, self.P+1):
            # Standard SMC sampler, we mutate the particles and then we resample
            ### Mutation step: 
            ###### Refresh velocities
            z[:, self.d:] = np.random.normal(loc=0.0, scale=1.0, size=(self.N, self.d))
            ###### Mutate positions 
            M = lambda z: THUG_MH(z, self.B, self.δ, self.log_ηs[n-1])
            Z = np.apply_along_axis(M, 1, z)
            ### Compute weights
            # Notice in this case the weight is different because we are not using the uniform kernel anymore
            # Importantly: this is now the INCREMENTAL weight and so has to be multiplied by the previous one.
            w_incremental = exp(np.apply_along_axis(self.log_ηs[n], 1, Z[:, :self.d]) - np.apply_along_axis(self.log_ηs[n-1], 1, Z[:, :self.d]))
#             w = (abs(np.apply_along_axis(f, 1, Z[:, :p]) - level_set_value) <= self.ϵs[n]).astype(float)
            w = self.WEIGHTS[n-1] * w_incremental
            w = w / w.sum()
            self.WEIGHTS[n] = w
            self.ESS[n]     = 1 / np.sum(w**2)
            ### Resample
            indeces = choice(a=np.arange(self.N), size=self.N, p=w)
            z = z[indeces, :]
            self.PARTICLES[n] = z
        self.total_time = time.time() - starting_time
        return z

In [44]:
MSTHUG_METROP = MultivariateMarkovSnippetsTHUGMetropolised(SETTINGS50)
zP_metrop = MSTHUG_METROP.sample()

/var/folders/31/dthljx8x5mldm2ll5_zz7xrr0000gn/T/ipykernel_63090/1779483409.py:82: RuntimeWarning: invalid value encountered in true_divide
  w = w / w.sum()


ValueError: probabilities contain NaN